In [3]:
!curl -fsSL https://deb.nodesource.com/setup_16.x | bash -
!apt-get install -y nodejs



                              DEPRECATION WARNING                            

     Node.js 16.x is no longer actively supported!

  You will not receive security or critical stability updates for this version.

  You should migrate to a supported version of Node.js as soon as possible.
  Use the installation script that corresponds to the version of Node.js you
  wish to install. e.g.
  
   * https://deb.nodesource.com/setup_16.x — Node.js 16 "Gallium" (deprecated)
   * https://deb.nodesource.com/setup_18.x — Node.js 18 "Hydrogen" (Maintenance)
   * https://deb.nodesource.com/setup_19.x — Node.js 19 "Nineteen" (deprecated)
   * https://deb.nodesource.com/setup_20.x — Node.js 20 LTS "Iron" (recommended)
   * https://deb.nodesource.com/setup_21.x — Node.js 21 "Iron" (current)
   


  Please see https://github.com/nodejs/Release for details about which
  version may be appropriate for you.

  The NodeSource Node.js distributions repository contains
  information both about supported ve

In [4]:
# Installing all the  dependencies required for the project
!pip install flask requests
!npm install express multer axios
!pip install pyngrok
!npm install cors
import os

# Create the folder if it doesn't exist
if not os.path.exists("uploads"):
    os.makedirs("uploads")
    print("Folder 'uploads' created successfully!")
else:
    print("Folder 'uploads' already exists!")



added 91 packages, and audited 92 packages in 4s

15 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities

added 1 package, and audited 93 packages in 748ms

15 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities
Folder 'uploads' created successfully!


In [5]:
from pyngrok import ngrok

NGROK_TOKEN = "2pemZzKwoD56bS2Xu0dFzYSZYOh_3wBgbW1WuuHfZhJigMbmW"
ngrok.set_auth_token(NGROK_TOKEN)

In [6]:
from pyngrok import ngrok

public_url = ngrok.connect(5000).public_url
print(f"ngrok URL: {public_url}")

ngrok URL: https://c6a1-34-46-78-134.ngrok-free.app


In [8]:
server_code = """
const express = require('express');
const multer = require('multer');
const axios = require('axios');
const fs = require('fs');
const path = require('path');
const cors = require("cors");

const app = express();
const PORT = 5000;

const NGROK_BASE_URL = "https://c6a1-34-46-78-134.ngrok-free.app/";

app.use(cors());

let storedFilePath ="https://c6a1-34-46-78-134.ngrok-free.app/content/uploads/1733143906149.pdf";

const storage = multer.diskStorage({
  destination: (req, file, cb) => {
    cb(null, './uploads'); // Destination folder for uploads
  },
  filename: (req, file, cb) => {
    // Extract file extension from the original file name
    const extname = path.extname(file.originalname).toLowerCase();
    const filename = Date.now() + extname; // Keep the original extension
    cb(null, filename); // Save the file with the new name
  },
});

const upload = multer({ storage: storage });

// Endpoint for file upload
app.post('/upload', upload.single('file'), (req, res) => {
  if (!req.file) {
    return res.status(400).send('No file uploaded.');
  }
  res.json({
    message: 'File uploaded successfully.',
    file: req.file,
  });
});

app.post('/query', express.json(), async (req, res) => {
    const { query } = req.body;

    if (!query) return res.status(400).send('Query is required.');
    if (!storedFilePath) return res.status(400).send('No file uploaded.');

    try {
        const fileContent = fs.readFileSync(storedFilePath, 'utf8');

        const response = await axios.post(
            `${NGROK_BASE_URL}/api/v1/query`,
            { context: fileContent, question: query },
            { headers: { 'Content-Type': 'application/json' } }
        );

        res.send(response.data);
    } catch (error) {
        console.error('Error querying model:', error.message);
        res.status(500).send('Error querying the model.');
    }
});
app.get('/test', (req, res) => {
    res.json({name:"Om"});
});

// Start the server
app.listen(PORT, () => {
    console.log(`Server running on http://localhost:${PORT}`);
});
"""

with open('server.js', 'w') as f:
    f.write(server_code)

In [22]:
!kill $(lsof -t -i:5000)

kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]


In [2]:
!node server.js &

node:internal/modules/cjs/loader:1143
  throw err;
  ^

Error: Cannot find module '/content/server.js'
    at Module._resolveFilename (node:internal/modules/cjs/loader:1140:15)
    at Module._load (node:internal/modules/cjs/loader:981:27)
    at Function.executeUserEntryPoint [as runMain] (node:internal/modules/run_main:128:12)
    at node:internal/main/run_main_module:28:49 {
  code: 'MODULE_NOT_FOUND',
  requireStack: []
}

Node.js v18.20.5
